In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
import pandas as pd 
import math
import random

import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.graphics.regressionplots import *
from sklearn import datasets, linear_model

In [18]:
Auto = pd.read_csv('/Users/shilpa/Documents/blog/Sharing_ISL_python/data/Auto.csv', header=0, na_values='?')
Auto = Auto.dropna().reset_index(drop=True) # drop the observation with NA values and reindex the obs from 0
Auto

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
0,18.0,8,307.0,130.0,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150.0,3433,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140.0,3449,10.5,70,1,ford torino
...,...,...,...,...,...,...,...,...,...
387,27.0,4,140.0,86.0,2790,15.6,82,1,ford mustang gl
388,44.0,4,97.0,52.0,2130,24.6,82,2,vw pickup
389,32.0,4,135.0,84.0,2295,11.6,82,1,dodge rampage
390,28.0,4,120.0,79.0,2625,18.6,82,1,ford ranger


In [49]:
np.random.seed(1)
train = np.random.choice(392, 196, replace=False)
select = np.in1d(range(Auto.shape[0]), train)

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
0,18.0,8,307.0,130.0,3504,12.0,70,1,chevrolet chevelle malibu
4,17.0,8,302.0,140.0,3449,10.5,70,1,ford torino
5,15.0,8,429.0,198.0,4341,10.0,70,1,ford galaxie 500
6,14.0,8,454.0,220.0,4354,9.0,70,1,chevrolet impala
8,14.0,8,455.0,225.0,4425,10.0,70,1,pontiac catalina
...,...,...,...,...,...,...,...,...,...
386,27.0,4,151.0,90.0,2950,17.3,82,1,chevrolet camaro
388,44.0,4,97.0,52.0,2130,24.6,82,2,vw pickup
389,32.0,4,135.0,84.0,2295,11.6,82,1,dodge rampage
390,28.0,4,120.0,79.0,2625,18.6,82,1,ford ranger


In [50]:
import statsmodels.formula.api as smf
lm = smf.ols('mpg~horsepower', data = Auto[select]).fit()
print(lm.summary())
preds = lm.predict(Auto)
square_error = (Auto['mpg'] - preds)**2
print('--------Test Error for 1st order--------')
print(np.mean(square_error[~select]))

                            OLS Regression Results                            
Dep. Variable:                    mpg   R-squared:                       0.620
Model:                            OLS   Adj. R-squared:                  0.618
Method:                 Least Squares   F-statistic:                     316.4
Date:                Sun, 06 Sep 2020   Prob (F-statistic):           1.28e-42
Time:                        01:40:04   Log-Likelihood:                -592.07
No. Observations:                 196   AIC:                             1188.
Df Residuals:                     194   BIC:                             1195.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     40.3338      1.023     39.416      0.0

0        2.495815
1        1.015656
2        2.602113
3        0.149687
4        0.966985
          ...    
387      0.156625
388    143.228052
389     25.770510
390      0.077415
391     14.116358
Length: 392, dtype: float64

In [52]:
lm2 = smf.ols ('mpg~horsepower + I(horsepower ** 2.0)', data = Auto[select]).fit()
preds = lm2.predict(Auto)
square_error = (Auto['mpg'] - preds)**2
print('--------Test Error for 2nd order--------')
print(square_error[~select].mean())

--------Test Error for 2nd order--------
20.252690858350043


## 5.3.2 Leave-One-Out Cross-Validation

In [56]:
from sklearn.model_selection import KFold, cross_val_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline

In [57]:
x = pd.DataFrame(Auto.horsepower)
y = Auto.mpg

model = LinearRegression()
model.fit(x, y)
print(model.intercept_)
print(model.coef_)

39.93586102117047
[-0.15784473]


In [63]:
k_fold = KFold(n_splits=x.shape[0]) # loo use folds equal to # of observations # Cool stuff
test = cross_val_score(model, x, y, cv=k_fold,  scoring = 'neg_mean_squared_error', n_jobs=-1)
print(np.mean(-test))

24.231513517929226


In [68]:
porder=2
model = Pipeline([('poly', PolynomialFeatures(degree=porder)), ('linear', LinearRegression())])
model

Pipeline(memory=None,
         steps=[('poly',
                 PolynomialFeatures(degree=2, include_bias=True,
                                    interaction_only=False, order='C')),
                ('linear',
                 LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
                                  normalize=False))],
         verbose=False)

In [72]:
A = []
for porder in range(1, 6):
    model = Pipeline([('poly', PolynomialFeatures(degree=porder)), ('linear', LinearRegression())])
    k_fold = KFold(n_splits=x.shape[0]) # loo use folds equal to # of observations
    test = cross_val_score(model, x, y, cv=k_fold,  scoring = 'neg_mean_squared_error', n_jobs=-1)
    A.append(np.mean(-test))
print(A)

[24.231513517929226, 19.248213124489745, 19.33498406411498, 19.424430307079398, 19.033198669299846]


## 5.3.4 The Bootstrap

In [106]:
Portfolio = pd.read_csv('/Users/shilpa/Documents/blog/Sharing_ISL_python/data/Portfolio.csv', header=0)
Portfolio.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 2 columns):
X    100 non-null float64
Y    100 non-null float64
dtypes: float64(2)
memory usage: 1.7 KB


In [128]:
def alpha_fn(data, lower_limit, upper_limit):
    X = data.X[lower_limit:upper_limit]
    Y = data.Y[lower_limit:upper_limit]
    return(np.var(Y) - np.cov(X,Y)[0,1])/(np.var(X) + np.var(Y) - 2 * np.cov(X, Y)[0,1])

Bootstrap- https://machinelearningmastery.com/a-gentle-introduction-to-the-bootstrap-method/

In [129]:
alpha_fn(Portfolio, 0,100)

0.5766511516104116